In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import re
import math
from matplotlib import pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
from fuzzywuzzy import process, fuzz
import warnings
import time
import random

warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

url_header = {
    'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36',
    'From': 'djslocum13@gmail.com'
}

C:\Users\Danny\Anaconda2\envs\threepointfive\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
class fantasy_football_draft():
    
    def __init__(self):
        self.draft_value = self.get_draft_value()
        self.draft_rankings = self.get_draft_rankings()
        self.draft_keeper_2019 = self.get_draft_keeper_2019()
        self.team_draft_values = self.get_team_draft_values()
        self.player_projections = self.get_player_projections()
        
    def get_draft_value(self):
        try:
            draft_value = pd.read_csv('draft_value.csv')
            print('-File Found (draft_value.csv)')
        except:
            draft_value = self.request_draft_pick_values()
            draft_value.to_csv('draft_value.csv', index=False)
            print('-Data Collected')
            print('-Data Saved')
        return draft_value
    
    def get_draft_rankings(self):
        try:
            draft_rankings = pd.read_csv('draft_rankings.csv')
            print('-File Found (draft_rankings.csv)')
        except:
            draft_rankings = self.request_draft_rankings_values()
            draft_rankings.to_csv('draft_value.csv', index=False)
            print('-Data Collected')
            print('-Data Saved')
        return draft_rankings

    def get_player_projections(self):
        try:
            player_projections = pd.read_csv('player_projections.csv')
            print('-File Found (player_projections.csv)')
        except:
            player_projections = self.request_player_projections()
            player_projections.to_csv('player_projections.csv', index=False)
            print('-Data Collected')
            print('-Data Saved')
        return player_projections
        
    def get_draft_keeper_2019(self):
        try:
            return pd.read_csv('2019_draft_keeper.csv')
        except:
            print('### Could not find file 2019_draft_keeper.csv ###')
            return None
        
    def request_draft_pick_values(self):
        print('-Scraping webpage for draft pick values')
        draft_pick_values_url = 'https://www.theringer.com/nfl/2018/8/20/17758898/fantasy-football-draft-pick-value-chart'
        response = requests.get(draft_pick_values_url, headers=url_header)
        if response.ok:
            soup = bs(response.text, 'html.parser')
            table = soup.find("table",{"class":"p-data-table"})
            header = [th.text for th in table.thead.tr.find_all("th")]
            values = pd.DataFrame([], columns=header)
            for index, tr in enumerate(table.tbody.find_all("tr")):
                vals = []
                for td in tr.find_all('td'):
                    vals.append(td.text)
                values.loc[index] = vals
            values['% of no. 1 pick'] = values['% of no. 1 pick'].str.replace('%', '')
            team = np.array([])
            for value in draft_value.values:
                mod8 = value[0] % 8
                mod16 = value[0] % 16
                if mod16 == 0:
                    val = 1.0
                elif mod8 == 0:
                    val = 8.0
                elif mod16 > 8:
                    val = 9 - mod8
                else:
                    val = mod8
                team = np.append(team, val)
            values['Team #'] = team
            values = values.astype({'Pick': 'int', 'Value': 'float', '% of no. 1 pick': 'float', 'Team #': 'int'})
            values['Round'] = values.Pick.apply(lambda x: math.ceil(x/8))
            # values.dtypes
        else:
            values = None
        return values
    def login_fantasy_pros(self):
        fantasypros_username = input("Input FantasyPros Username: ")
        fantasypros_password = input("Input FantasyPros Password: ")
        payload = {
            'csrfmiddlewaretoken': '5L5My2xkICf60HEvTdwU9rmmopM1ltviYeG5SVoOojVVWGQvA3eumMvCel9szylr',
            'username': fantasypros_username,
            'password': fantasypros_password,
            'import_id': ''
        }
        session = requests.Session()
        print('-Scraping FantasyPros')
        fantasy_pros_login = 'https://secure.fantasypros.com/accounts/login/'
        post = session.post(fantasy_pros_login, data=payload, headers=url_header)
        return session
    
    def request_draft_rankings_values(self):
        session = self.login_fantasy_pros()
        fantasy_pros_rankings_url = 'https://www.fantasypros.com/nfl/rankings/ppr-cheatsheets.php?filters=35fd0c3d9a' 
        response = session.get(fantasy_pros_rankings_url, headers=url_header)
        values = None
        if response.ok:
            soup = bs(response.text, 'html.parser')
            table = soup.find('table', {'id': 'rank-data'})
            header = [th.text for th in table.thead.find_all('th')]
            header.append('Tier')
            header.append('Team')
            values = pd.DataFrame([], columns=header)
            for index, tr in enumerate(table.tbody.find_all("tr")):
                if 'tier-row' in tr.get('class'):
                    tier = tr.get('data-tier')
                elif 'static' in tr.get('class'):
                    continue
                elif 'player-row' in tr.get('class'):
                    vals = []
                    for i, td in enumerate(tr.find_all('td')):
                        if i == 2:
                            team = td.small.text
                            vals.append(td.span.text)
                        else:
                            vals.append(td.text)
                    vals.append(None)
                    vals.append(tier)
                    vals.append(team)
                    values.loc[index] = vals
            values = values.drop(['WSID','Notes\r\n                            '], axis=1)
            values.Pos = values.Pos.apply(lambda x: re.sub('\d', '', x))
            values['vs. ADP'] = values['vs. ADP'].apply(lambda x: re.sub('[+]', '', x))
            values.Team[values.Team == ''] = values[values.Team == '']['Overall (Team)'].apply(lambda x: re.findall(r'(?:[A-Za-z]+\s*[(])([A-Za-z]+)', x)[0])
            values.ADP = values.ADP.str.replace(',', '')
            values.Bye[values.Bye == ''] = 999
            values = values.astype({
                'Rank': 'int',
                'Overall (Team)': 'str',
                'Pos': 'category',
                'Bye': 'int',
                'Best': 'int',
                'Worst': 'int',
                'Avg': 'float',
                'Std Dev': 'float',
                'ADP': 'float',
                'vs. ADP': 'float',
                'Tier': 'int',
                'Team': 'category'
            })
            #values.sample(20)
            #values.dtypes
        else:
            print("### Error connecting to FantasyPros ###")
        return values
    
    def get_team_draft_values(self):
        draft_value = self.draft_value
        draft_keeper_2019 = self.draft_keeper_2019
        team_draft_values = draft_value.merge(draft_keeper_2019[['Order', 'Team']], how='left', left_on='Team #', right_on='Order')
        team_draft_values = team_draft_values[team_draft_values.Pick <= 128]
        return team_draft_values[['Pick','Round','Order','Team','Value','% of no. 1 pick']]
    
    def show_team_draft_values(self):
        team_draft_values = self.team_draft_values
        team_draft_values_sum = team_draft_values.groupby(['Team']).Value.sum().reset_index().sort_values(['Value'], ascending=False).reset_index().drop(['index'], axis=1)
        print(team_draft_values_sum)
        y_val = team_draft_values_sum['Value']
        x_val = np.arange(len(y_val))
        x_names = team_draft_values_sum['Team']
        self.plot_bar(x_val, y_val, x_names, "Total Draft Value", "Draft Value")
        
    def plot_bar(self, x, y, x_labels=[], y_label='', title=''):
        plt.figure(figsize=(5,3))
        plt.bar(x, y, align='center')
        if len(x_labels) > 0:
            plt.xticks(x, x_labels)
            plt.xticks(rotation=90)
        if y_label != '':
            plt.ylabel(y_label)
        if title != '':
            plt.title(title)
        plt.show()
    
    def request_player_stats(self, year):
        stats_2018 = 'https://www.fantasypros.com/nfl/stats/{}.php?year={}&scoring=PPR'
        
    def request_player_projections(self):
        player_projections = player_projections_data('draft')
        return player_projections.data      

In [3]:
class player_projections_data():
    def __init__(self, week):
        self.data = pd.DataFrame([])
        self.week = week
        self.positions = ['qb','rb','wr','te','k','dst']
        self.position_requests()
        
    def position_requests(self):
        session = requests.Session()
        print('-Scraping FantasyPros')
        positions = self.positions
        for pos in positions:
            fantasy_pros_projections_url = 'https://www.fantasypros.com/nfl/projections/{}.php?scoring=PPR&week={}'
            response = session.get(fantasy_pros_projections_url.format(pos, self.week), headers=url_header)
            values = None
            if response.ok:
                soup = bs(response.text, 'html.parser')
                data = self.get_data(soup)
                data['Position'] = pos
                self.data = self.data.append(data)
                wait = random.randint(400,600)/100
                print('   -Scraping for {} position projections, wait {} seconds'.format(pos, int(wait)))
                time.sleep(wait)
            else:
                print("### Error connecting to FantasyPros for the {} position ###".format(pos))
        print("-Done scraping position projections")
        self.data = self.data.reset_index()
        
    def get_data(self, soup):
        table = soup.find('table', {'class': 'table'})
        values = self.get_dataframe_header(table)
        for index, tr in enumerate(table.tbody.find_all("tr")):
            vals = []
            for i, td in enumerate(tr.find_all('td')):
                if i == 0:
                    name = td.find('a', {'class': 'player-name'}).text
                    vals.append(name)
                else:
                    vals.append(td.text)
            values.loc[index] = vals
        values = values.replace({',': ''}, regex=True)
        return values
    
    def get_dataframe_header(self, table):
        tr = table.thead.find_all('tr')
        if len(tr) == 2:
            thead_top = tr[0].find_all('td')
            thead_top_header = []
            for td in thead_top:
                col_num = td.get("colspan")
                if col_num == None:
                    col_num = 1
                thead_top_header += int(col_num) * [td.text]
            header = [thead_top_header[index] + ' ' + th.text for index, th in enumerate(tr[1].find_all('th'))]
        elif len(tr) == 1:
            header = [th.text for th in table.thead.find_all('th')]
        else: 
            print('### No header found ###')
        header[0] = "Player"
        values = pd.DataFrame([], columns=header)
        return values


In [4]:
class fantasy_football_keeper(fantasy_football_draft):
    
    def __init__(self):
        super().__init__()
        self.roster_2018 = self.get_roster_2018()
        self.keeper_current_values = self.get_keeper_current_values()
        self.keeper_full_values = self.get_keeper_full_values()
        self.keeper_trade_values = self.get_keeper_trade_values()
        
    def get_roster_2018(self):
        try:
            data = pd.read_csv('2018_roster.csv')
            return data
        except:
            print('### Could not find file 2018_roster.csv ###')
            return None 

    def get_keeper_current_values(self):
        roster_2018 = self.roster_2018
        draft_rankings = self.draft_rankings
        draft_rankings = draft_rankings.drop(['Best','Worst','Bye'], axis=1)
        draft_rankings['Overall (Team)'] = draft_rankings['Overall (Team)'].str.lower()
        team_draft_values = self.team_draft_values
        roster_2018['name_validated'] = self.correlate_player_names()
        self.roster_2018 = roster_2018
        roster_2018 = roster_2018.drop(['name','drafted round','position'], axis=1) 
        keeper_current_values = roster_2018.merge(draft_rankings, how='left', left_on='name_validated', right_on='Overall (Team)')
        keeper_current_values = keeper_current_values[keeper_current_values.Rank.notnull()]
        keeper_current_values['Rank'] = keeper_current_values['Rank'].astype('int')
        keeper_current_values = keeper_current_values.merge(self.draft_value, how='left', left_on='Rank', right_on='Pick')
        keeper_current_values = keeper_current_values.drop(['Overall (Team)'], axis=1)
        return keeper_current_values
        
    def get_keeper_full_values(self):
        keeper_full_values = self.keeper_current_values.merge(self.team_draft_values[['Team', 'Round', 'Value', 'Pick', '% of no. 1 pick']], how='left', left_on=['team', 'keeper round'], right_on=['Team', 'Round'])
        keeper_full_values = keeper_full_values.drop(['Team_y', 'Round_y'], axis=1)
        keeper_full_values[['name_validated','Pos','Team_x','team', 'keeper round','Pick_y','Value_y','% of no. 1 pick_y','Avg', 'Round_x', 'Rank', 'Value_x', '% of no. 1 pick_x']]
        keeper_full_values = keeper_full_values.dropna()
        keeper_full_values['Value_add'] = keeper_full_values['Value_x'] - keeper_full_values['Value_y']
        keeper_full_values['Value_add_norm'] = keeper_full_values['% of no. 1 pick_x'] - keeper_full_values['% of no. 1 pick_y']
        keeper_full_values.Round_x = keeper_full_values.Round_x.astype('int')
        keeper_full_values = keeper_full_values[['name_validated','Pos','Team_x','team', 'keeper round', 'Round_x', 'Value_add', 'Value_add_norm']]
        return keeper_full_values.sort_values(['Value_add'], ascending=False)

    def get_keeper_trade_values(self):
        keeper_trade_values = pd.DataFrame([])
        keeper_current_values = self.keeper_current_values
        best_keeper_value = self.get_best_keeper_value()
        my_keepers = keeper_current_values[keeper_current_values.team == 'Danny']
        for team in self.draft_keeper_2019.Team:
            if team != "Danny":
                my_keepers.team = team
                full_merge = my_keepers.merge(self.team_draft_values[['Team', 'Round', 'Value', 'Pick', '% of no. 1 pick']], how='left', left_on=['team', 'keeper round'], right_on=['Team', 'Round'])
                full_merge = full_merge.drop(['Team_y', 'Round_y'], axis=1)
                full_merge = full_merge.dropna()
                full_merge['Value_add'] = full_merge['Value_x'] - full_merge['Value_y']
                full_merge['Value_add_norm'] = full_merge['% of no. 1 pick_x'] - full_merge['% of no. 1 pick_y']
                full_merge.Round_x = full_merge.Round_x.astype('int')
                full_merge = full_merge[['name_validated','Pos','Team_x','team', 'keeper round', 'Round_x', 'Value_add', 'Value_add_norm']]
                full_merge['trade_value'] = full_merge.Value_add - float(best_keeper_value.Value_add[best_keeper_value.team == team])
                keeper_trade_values = keeper_trade_values.append(full_merge, ignore_index=True)
        keeper_trade_values = keeper_trade_values[['name_validated','team','trade_value']].sort_values(['trade_value'], ascending=False)[keeper_trade_values.trade_value > 0]
        draft_value = self.draft_value
        keeper_trade_values['trade_round'] = keeper_trade_values.trade_value.apply(lambda x: int(draft_value.Round[draft_value.Value == min(draft_value.Value, key=lambda y:abs(y-x))]))
        return keeper_trade_values
    
    def correlate_player_names(self):
        name_validated = np.array([])
        roster_2018_players = self.roster_2018.name.str.lower()
        fantasy_pros_players = self.draft_rankings['Overall (Team)'].str.lower()
        for name in roster_2018_players:
            estimate = process.extractOne(name, fantasy_pros_players, scorer=fuzz.ratio)
            est1 = estimate[0]
            est_per = estimate[1]
            if (est_per < 100) & (est_per >= 85):
                #print('Name = {}, Estimated = {}, Percent = {}'.format(name, est1, est_per))
                name_validated = np.append(name_validated, est1)
            else:
                name_validated = np.append(name_validated, name)
        return name_validated

    def get_best_keeper_value(self):
        best_keeper_value = self.keeper_full_values.groupby(['team'])['Value_add'].max().reset_index()
        return best_keeper_value
    
    def visualize_keeper_full_values(self):
        keeper_full_values = self.keeper_full_values
        keeper_full_values_viz = keeper_full_values[keeper_full_values.Value_add > 0].sort_values(['team','Value_add'], ascending=False)
        y_val = keeper_full_values_viz['Value_add']
        x_val = np.arange(len(y_val))
        x_names = keeper_full_values_viz['name_validated']
        colors = {
            'Danny':'blue',
            'Zack': 'red',
            'Hunter': 'orange',
            'Nate': 'yellow',
            'Brendan': 'green',
            'Mac': 'purple',
            'Vinay': 'pink',
            'Bryan': 'black'
        }
        color = [colors[val] for val in keeper_full_values_viz.team]
        plt.figure(figsize=(20,10))
        plt.bar(x_val, y_val, align='center', color=color)
        plt.xticks(x_val, x_names)
        plt.xticks(rotation=90)
        plt.ylabel('Keeper Value')
        plt.title('Keeper Analysis')
        plt.show()
        

In [5]:
test = fantasy_football_keeper()

-File Found (draft_value.csv)
-File Found (draft_rankings.csv)
-File Found (player_projections.csv)


In [6]:
test.player_projections.reset_index()

,index,FF,FG,FGA,FPTS,FR,INT,MISC FL,MISC FPTS,PA,PASSING ATT,PASSING CMP,PASSING INTS,PASSING TDS,PASSING YDS,Player,Position,RECEIVING REC,RECEIVING TDS,RECEIVING YDS,RUSHING ATT,RUSHING TDS,RUSHING YDS,SACK,SAFETY,TD,XPT,YDS AGN
0,0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,338.2,NaN,581.5,379.9,13.3,36.0,4779.6,Patrick Mahomes,qb,NaN,NaN,NaN,52.4,2.0,219.6,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,2.7,320.7,NaN,511.7,338.0,12.0,26.4,4073.5,Deshaun Watson,qb,NaN,NaN,NaN,95.2,4.4,552.2,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,2.1,308.6,NaN,590.0,375.0,7.6,29.5,4284.4,Aaron Rodgers,qb,NaN,NaN,NaN,46.2,2.1,260.5,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,2.0,302.1,NaN,584.8,394.6,10.6,30.0,4632.4,Matt Ryan,qb,NaN,NaN,NaN,48.2,1.5,132.8,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,1.7,300.1,NaN,520.4,339.8,13.8,24.4,3808.7,Cam Newton,qb,NaN,NaN,NaN,112.9,4.4,545.0,NaN,NaN,NaN,NaN,NaN
5,5,NaN,NaN,NaN,NaN,NaN,NaN,2.4,296.5,NaN,577.0,379.3,13.7,31.9,4329.3,Andrew Luck,qb,NaN,NaN,NaN,49.7,1.2,210.6,NaN,NaN,NaN,NaN,NaN
6,6,NaN,NaN,NaN,NaN,NaN,NaN,1.6,290.2,NaN,539.0,387.6,8.8,30.4,4342.6,Drew Brees,qb,NaN,NaN,NaN,24.7,1.9,40.1,NaN,NaN,NaN,NaN,NaN
7,7,NaN,NaN,NaN,NaN,NaN,NaN,2.7,288.0,NaN,564.7,358.8,16.6,27.7,4454.9,Jameis Winston,qb,NaN,NaN,NaN,57.2,1.6,277.0,NaN,NaN,NaN,NaN,NaN
8,8,NaN,NaN,NaN,NaN,NaN,NaN,5.2,286.7,NaN,436.2,258.9,10.8,17.2,3016.9,Lamar Jackson,qb,NaN,NaN,NaN,177.5,6.4,911.0,NaN,NaN,NaN,NaN,NaN
9,9,NaN,NaN,NaN,NaN,NaN,NaN,2.2,286.6,NaN,550.3,351.5,11.7,30.5,4372.2,Jared Goff,qb,NaN,NaN,NaN,31.1,1.2,102.9,NaN,NaN,NaN,NaN,NaN


In [ ]:
'https://www.fantasypros.com/nfl/stats/qb.php?year=2018&scoring=PPR&ownership=consensus&range=week&week=1'
'https://www.fantasypros.com/nfl/stats/qb.php?year=2018&scoring=PPR&ownership=consensus&range=full'

In [1]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget
from PyQt5.QtGui import QIcon

In [2]:
class App(QWidget):

    def __init__(self):
        super().__init__()
        self.title = 'PyQt5 simple window - pythonspot.com'
        self.left = 150
        self.top = 150
        self.width = 640
        self.height = 480
        self.initUI()
        
    def initUI(self):
        self.setWindowTitle(self.title)
        self.setGeometry(self.left, self.top, self.width, self.height)
        self.show()
    
if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = App()
    sys.exit(app.exec_())

SystemExit: 0

C:\Users\Danny\Anaconda2\envs\threepointfive\lib\site-packages\IPython\core\interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
